# Import statements
Following packages are necessary:

In [1]:
import random
import utils
import closs
import os
import imageio
import json
from lastengen import LASTENSequence
import importlib
from glob import glob

import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as kb
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint

from unet import UNet
from unet import preprocess_input as pre_une

from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from pdb import set_trace

# Checks
The version of tensorflow as well as the GPU support are checked.

In [2]:
print(tf.__version__)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.get_visible_devices('GPU')

2.2.0-rc3


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  # Disable all GPUS
  tf.config.set_visible_devices([], 'GPU')
  visible_devices = tf.config.get_visible_devices()
  for device in visible_devices:
    assert device.device_type != 'GPU'
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

print(tf.config.get_visible_devices('GPU'))
print(tf.config.get_visible_devices('CPU'))

[]
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


# Definition
Size definition is done here

In [4]:
width = 384
height = 384

grid_width = 18
grid_height = 18

grid_points = grid_width * grid_height

# Functionality
Necessary funcionality is added here

In [5]:
def eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path):
    test_gen = LASTENSequence(path_test,
                               path_fixed,
                               batch_size=1,
                               width=width,
                               height=height,
                               grid_width=grid_width, 
                               grid_height=grid_height,
                               preprocess_input=pre_une,
                               shuffle=True,
                               label="keypoints",
                               channel="moving+fixed")

    validation_gen = LASTENSequence(path_validation,
                               path_fixed,
                               batch_size=1,
                               width=width,
                               height=height,
                               grid_width=grid_width, 
                               grid_height=grid_height,
                               preprocess_input=pre_une,
                               shuffle=False,
                               label="keypoints",
                               channel="moving+fixed")

    eu_loss = closs.EuclideanLoss(batch_size=1, grid_width=grid_width, grid_height=grid_height, loss_type='maed')

    model.compile(optimizer='adam', loss = eu_loss)
    model.load_weights(store_path+'/weights.100.hdf5')

    warp_val = dict()
    maed_val_array = np.zeros(len(validation_gen))
    for index, val in enumerate(validation_gen):
        image_id = validation_gen._image_ids[index]
        
        if index % 100 == 0:
            print(index)
        X, y = val
        maed = model.evaluate(X,y,verbose=0)
        maed_val_array[index] = maed

        y_pred = model.predict(X)
        u_x = y_pred[0,:,:,0]
        u_y = y_pred[0,:,:,1]

        for inner_index in range(0,grid_points):
            x_pos = int(y[0, inner_index, 0, 0])
            y_pos = int(y[0, inner_index, 1, 0])

            ux_field = y_pred[0,:,:,0]
            uy_field = y_pred[0,:,:,1]

            ux = ux_field[y_pos][x_pos]
            uy = uy_field[y_pos][x_pos]

            x_pos = int(round(x_pos + ux))
            y_pos = int(round(y_pos + uy))

            warp_val[str(inner_index)] = [x_pos, y_pos]
            with open(store_path +'/val/{}_w.json'.format(image_id), 'w') as fp:
                json.dump(warp_val, fp)    

    warp_test = dict()
    maed_test_array = np.zeros(len(test_gen))
    for index, val in enumerate(test_gen):
        image_id = test_gen._image_ids[index]
        
        if index % 100 == 0:
            print(index)
        X, y = val
        maed = model.evaluate(X,y,verbose=0)
        maed_test_array[index] = maed
        
        y_pred = model.predict(X)
        u_x = y_pred[0,:,:,0]
        u_y = y_pred[0,:,:,1]

        for inner_index in range(0,grid_points):
            x_pos = int(y[0, inner_index, 0, 0])
            y_pos = int(y[0, inner_index, 1, 0])

            ux_field = y_pred[0,:,:,0]
            uy_field = y_pred[0,:,:,1]

            ux = ux_field[y_pos][x_pos]
            uy = uy_field[y_pos][x_pos]

            x_pos = int(round(x_pos + ux))
            y_pos = int(round(y_pos + uy))

            warp_test[str(inner_index)] = [x_pos, y_pos]
            with open(store_path +'/test/{}_w.json'.format(image_id), 'w') as fp:
                json.dump(warp_test, fp)

    maed_array = np.concatenate((maed_val_array, maed_test_array))
    val_set = ['val' for i in range(len(validation_gen))]
    test_set = ['test' for i in range(len(test_gen))]
    set_type = val_set + test_set
    image_id = validation_gen._image_ids + test_gen._image_ids

    dataset = pd.DataFrame({'MAED': maed_array, 'Set': set_type, 'Image': image_id})
    dataset.to_csv(store_path + '/evaluation.csv')

In [6]:
def eval_nearest_neighbor(experiment_2_set_2_image_2_accuracy):
    accuracys = []
    images = []
    set_types = []
    experiments = []
    
    for experiment, set_2_image_2_accuracy in experiment_2_set_2_image_2_accuracy.items():
        for set_type, image_2_accuracy in set_2_image_2_accuracy.items():
            for image, accuracy in image_2_accuracy.items():
                accuracys.append(accuracy)
                images.append(image)
                set_types.append(set_type)
                experiments.append(experiment)
                
    dataset = pd.DataFrame({'Accuracy': accuracys, 'Image': images, 'Set': set_types, 'Experiment': experiments})
    dataset.to_csv('experiments/7_euclidean/evaluation_nn.csv')

In [7]:
def spatial_distribution(store_path, path_fixed):
    x_val = []
    y_val = []
    image = []
    point = []
    set_types = []
    
    for set_type in ['val', 'test']:
        data_path = os.path.join(store_path, set_type)
        globs = glob(data_path + os.sep + "*_w.json")
        globs = [int(path.split(os.sep)[-1].split(".")[0].split("_")[0]) for path in globs]
        image_ids = sorted(globs)
        
        for image_id in image_ids:
            # 0)Load data
            warp_path = data_path + os.sep + "{}_w.json".format(image_id)
            with open(warp_path) as warped_file:
                warped_json = json.load(warped_file)

            # 1) Sort out all obsolete points
            for key, value in list(warped_json.items()):
                if value[0] < 2.0 and value[1] < 2.0:
                    _ = warped_json.pop(key)

            # 2) Build arrays
            for key, value in list(warped_json.items()):
                x_val.append(value[0])
                y_val.append(value[1])
                image.append(image_id)
                point.append(key)
                set_types.append(set_type)
            
    # 3) Build dataframe
    dataset = pd.DataFrame({'x': x_val, 'y': y_val, 'Set': set_types, 'Image': image, 'Point': point})
    dataset.to_csv(store_path + '/evaluation_spatial.csv')

In [8]:
def spatial_display(store_path, path_validation, path_test):
    x_val = []
    y_val = []
    image = []
    point = []
    set_types = []
    image_types = []
    
    # Warp
    for set_type in ['val', 'test']:
        data_path = os.path.join(store_path, set_type)
        globs = glob(data_path + os.sep + "*_w.json")
        globs = [int(path.split(os.sep)[-1].split(".")[0].split("_")[0]) for path in globs]
        image_ids = sorted(globs)
        
        for image_id in image_ids:
            # 0)Load data
            warp_path = data_path + os.sep + "{}_w.json".format(image_id)
            with open(warp_path) as warped_file:
                warped_json = json.load(warped_file)

            # 1) Sort out all obsolete points
            for key, value in list(warped_json.items()):
                if value[0] < 2.0 and value[1] < 2.0:
                    _ = warped_json.pop(key)

            # 2) Build arrays
            for key, value in list(warped_json.items()):
                x_val.append(value[0])
                y_val.append(value[1])
                image.append(image_id)
                point.append(key)
                set_types.append(set_type)
                image_types.append('warped')
                
    # Moving validation
    globs = glob(path_validation + os.sep + "*.json")
    globs = [int(path.split(os.sep)[-1].split(".")[0].split("_")[0]) for path in globs]
    image_ids = sorted(globs)
    
    for image_id in image_ids:
        # 0)Load data
        moving_path = path_validation + os.sep + "{}.json".format(image_id)
        with open(moving_path) as moving_file:
            moving_json = json.load(moving_file)

        # 1) Sort out all obsolete points
        for key, value in list(moving_json.items()):
            if value[0] < 2.0 and value[1] < 2.0:
                _ = warped_json.pop(key)

        # 2) Build arrays
        for key, value in list(moving_json.items()):
            x_val.append(value[0])
            y_val.append(value[1])
            image.append(image_id)
            point.append(key)
            set_types.append('val')
            image_types.append('moving')
            
    # Moving test
    globs = glob(path_test + os.sep + "*.json")
    globs = [int(path.split(os.sep)[-1].split(".")[0].split("_")[0]) for path in globs]
    image_ids = sorted(globs)

    for image_id in image_ids:
        # 0)Load data
        moving_path = path_test + os.sep + "{}.json".format(image_id)
        with open(moving_path) as moving_file:
            moving_json = json.load(moving_file)

        # 1) Sort out all obsolete points
        for key, value in list(moving_json.items()):
            if value[0] < 2.0 and value[1] < 2.0:
                _ = warped_json.pop(key)

        # 2) Build arrays
        for key, value in list(moving_json.items()):
            x_val.append(value[0])
            y_val.append(value[1])
            image.append(image_id)
            point.append(key)
            set_types.append('test')
            image_types.append('moving')
            
    # Build dataframe
    dataset = pd.DataFrame({'x': x_val, 'y': y_val, 'Set': set_types, 'Image': image, 'Point': point, 'Type': image_types})
    dataset.to_csv(store_path + '/evaluation_display.csv')

In [9]:
experiment_2_set_2_image_2_accuracy = dict()

# 7.1) Supervised Euclidean for LASTEN with fixed image BS=2

In [10]:
experiment = 1
store_path = 'experiments/7_{}_euclidean'.format(experiment)

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/fix'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/test'

model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(width, height, 2))
eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path)

accuracy_val = utils.nearest_neighbor(store_path, path_fixed, 'val', scale_factor=0.5)
accuracy_test = utils.nearest_neighbor(store_path, path_fixed, 'test', scale_factor=0.5)

set_2_image_2_accuracy = dict()
set_2_image_2_accuracy['val'] = accuracy_val
set_2_image_2_accuracy['test'] = accuracy_test
experiment_2_set_2_image_2_accuracy[experiment] = set_2_image_2_accuracy

#spatial_distribution(store_path, path_fixed)
spatial_display(store_path, path_validation, path_test)

0
0


# 7.2) Supervised Euclidean for LASTEN with fixed image BS=4

In [11]:
experiment = 2
store_path = 'experiments/7_{}_euclidean'.format(experiment)

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/fix'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/test'

model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(width, height, 2))
eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path)

accuracy_val = utils.nearest_neighbor(store_path, path_fixed, 'val', scale_factor=0.5)
accuracy_test = utils.nearest_neighbor(store_path, path_fixed, 'test', scale_factor=0.5)

set_2_image_2_accuracy = dict()
set_2_image_2_accuracy['val'] = accuracy_val
set_2_image_2_accuracy['test'] = accuracy_test
experiment_2_set_2_image_2_accuracy[experiment] = set_2_image_2_accuracy

#spatial_distribution(store_path, path_fixed)
spatial_display(store_path, path_validation, path_test)

0
0


# 7.3) Supervised Euclidean for LASTENS with fixed image

In [16]:
experiment = 3
store_path = 'experiments/7_{}_euclidean'.format(experiment)

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTENS/fix'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTENS/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTENS/validation'
path_validation2 = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTENS/test'
path_test2 = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/test'

model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(width, height, 2))
#eval_pred(model, path_train, path_fixed, path_validation, path_test2, store_path)

#accuracy_val = utils.nearest_neighbor(store_path, path_fixed, 'val', scale_factor=0.5)
#accuracy_test = utils.nearest_neighbor(store_path, path_fixed, 'test', scale_factor=0.5)

#set_2_image_2_accuracy = dict()
#set_2_image_2_accuracy['val'] = accuracy_val
#set_2_image_2_accuracy['test'] = accuracy_test
#experiment_2_set_2_image_2_accuracy[experiment] = set_2_image_2_accuracy

#spatial_distribution(store_path, path_fixed)
spatial_display(store_path, path_validation, path_test2)

# 7.4) Supervised Euclidean for LASTENA with fixed image

In [17]:
experiment = 4
store_path = 'experiments/7_{}_euclidean'.format(experiment)

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTENA/fix'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTENA/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTENA/validation'
path_validation2 = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTENA/test'
path_test2 = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/test'

model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(width, height, 2))
#eval_pred(model, path_train, path_fixed, path_validation, path_test2, store_path)

#accuracy_val = utils.nearest_neighbor(store_path, path_fixed, 'val', scale_factor=0.5)
#accuracy_test = utils.nearest_neighbor(store_path, path_fixed, 'test', scale_factor=0.5)

#set_2_image_2_accuracy = dict()
#set_2_image_2_accuracy['val'] = accuracy_val
#set_2_image_2_accuracy['test'] = accuracy_test
#experiment_2_set_2_image_2_accuracy[experiment] = set_2_image_2_accuracy

#spatial_distribution(store_path, path_fixed)
spatial_display(store_path, path_validation, path_test2)

# 7.5) Supervised Euclidean for LASTENAS with fixed image

In [18]:
experiment = 5
store_path = 'experiments/7_{}_euclidean'.format(experiment)

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTENAS/fix'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTENAS/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTENAS/validation'
path_validation2 = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTENAS/test'
path_test2 = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/test'

model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(width, height, 2))
#eval_pred(model, path_train, path_fixed, path_validation, path_test2, store_path)

#accuracy_val = utils.nearest_neighbor(store_path, path_fixed, 'val', scale_factor=0.5)
#accuracy_test = utils.nearest_neighbor(store_path, path_fixed, 'test', scale_factor=0.5)

#set_2_image_2_accuracy = dict()
#set_2_image_2_accuracy['val'] = accuracy_val
#set_2_image_2_accuracy['test'] = accuracy_test
#experiment_2_set_2_image_2_accuracy[experiment] = set_2_image_2_accuracy

#spatial_distribution(store_path, path_fixed)
spatial_display(store_path, path_validation, path_test2)

In [15]:
eval_nearest_neighbor(experiment_2_set_2_image_2_accuracy)